In [1]:
# 他人图片集下载：http://vis-www.cs.umass.edu/lfw/lfw.tgz

import tensorflow as tf
from img_to_tf_record import Img2TFRecord


my_faces_path = './my_faces'
other_faces_path = './other_faces'
size = 64

imgs = []
labs = []


x = tf.placeholder(tf.float32, [None, 250, 151, 1])  #[None, size, size, 3] Height, Width, channel
y_ = tf.placeholder(tf.float32, [None, 2])

keep_prob_5 = tf.placeholder(tf.float32)
keep_prob_75 = tf.placeholder(tf.float32)

def weightVariable(shape):
    init = tf.random_normal(shape, stddev = 0.01)
    return tf.Variable(init)

def biasVariable(shape):
    init = tf.random_normal(shape)
    return tf.Variable(init)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def maxPool(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

def dropout(x, keep):
    return tf.nn.dropout(x, keep)

def cnnLayer():
    # 第一层
    W1 = weightVariable([3, 3, 3, 32]) # 卷积核大小(3,3)， 输入通道(3)， 输出通道(32)
    b1 = biasVariable([32])
    # 卷积
    conv1 = tf.nn.relu(conv2d(x, W1) + b1)
    # 池化
    pool1 = maxPool(conv1)
    # 减少过拟合，随机让某些权重不更新
    drop1 = dropout(pool1, keep_prob_5)

    # 第二层
    W2 = weightVariable([3, 3, 32, 64])
    b2 = biasVariable([64])
    conv2 = tf.nn.relu(conv2d(drop1, W2) + b2)
    pool2 = maxPool(conv2)
    drop2 = dropout(pool2, keep_prob_5)

    # 第三层
    W3 = weightVariable([3, 3, 64, 64])
    b3 = biasVariable([64])
    conv3 = tf.nn.relu(conv2d(drop2, W3) + b3)
    pool3 = maxPool(conv3)
    drop3 = dropout(pool3, keep_prob_5)

    # 全连接层
    Wf = weightVariable([8 * 8 * 64, 512])
    bf = biasVariable([512])
    drop3_flat = tf.reshape(drop3, [-1, 8 * 8 * 64])
    dense = tf.nn.relu(tf.matmul(drop3_flat, Wf) + bf)
    dropf = dropout(dense, keep_prob_75)

    # 输出层
    Wout = weightVariable([512, 2])
    bout = weightVariable([2])
    #out = tf.matmul(dropf, Wout) + bout
    out = tf.add(tf.matmul(dropf, Wout), bout)
    return out

def cnnTrain():
    out = cnnLayer()
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out, labels = y_))
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    # 比较标签是否相等，再求的所有数的平均值，tf.cast(强制转换类型)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out, 1), tf.argmax(y_, 1)), tf.float32))
    # 将loss与accuracy保存以供tensorboard使用
    tf.summary.scalar('loss', cross_entropy)
    tf.summary.scalar('accuracy', accuracy)
    merged_summary_op = tf.summary.merge_all()
    # 数据保存器的初始化
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter('./tmp', graph = tf.get_default_graph())
        for n in range(10):
             # 每次取128(batch_size)张图片
            for i in range(num_batch):
                batch_x = train_x[i*batch_size : (i + 1) * batch_size]
                batch_y = train_y[i*batch_size : (i + 1) * batch_size]
                # 开始训练数据，同时训练三个变量，返回三个数据
                _, loss, summary = sess.run([train_step, cross_entropy, merged_summary_op],
                                           feed_dict = {x:batch_x, y_:batch_y, keep_prob_5:0.5, keep_prob_75:0.75})
                summary_writer.add_summary(summary, n * num_batch + i)
                # 打印损失
                # print("loss ", n*num_batch + i, loss)

                if (n * num_batch + i) % 100 == 0:
                    # 获取测试数据的准确率
                    acc = accuracy.eval({x:test_x, y_:test_y, keep_prob_5:1.0, keep_prob_75:1.0})
                    print(n * num_batch + i, "acc:", acc, "  loss:", loss)
                    # 准确率大于0.98时保存并退出
                    if acc > 0.98 and n > 2:
                        saver.save(sess, model_path + '/train_faces.model', global_step = n * num_batch + i)
                        sys.exit(0)
        print('accuracy less 0.98, exited!')


def is_my_face(image):  
    res = sess.run(predict, feed_dict={x: [image/255.0], keep_prob_5:1.0, keep_prob_75: 1.0})  
    if res[0] == 1:  
        return True  
    else:  
        return False  


if __name__ == '__main__':
    one_Set = Img2TFRecord('./imagenet-dogs', './out/dogs_record')

    # test renerate function
    # one_Set.generate_tf_record_files( (250, 151) )
    
    
    # cnnTrain()

breed = n02107574-Greater_Swiss_Mountain_dog, label = 0

breed = n02099267-flat-coated_retriever, label = 1

breed = n02105641-Old_English_sheepdog, label = 2

breed = n02096437-Dandie_Dinmont, label = 3

breed = n02111277-Newfoundland, label = 4

breed = n02100877-Irish_setter, label = 5

breed = n02101556-clumber, label = 6

breed = n02088466-bloodhound, label = 7

breed = n02089973-English_foxhound, label = 8

breed = n02106030-collie, label = 9

breed = n02106662-German_shepherd, label = 10

breed = n02108422-bull_mastiff, label = 11

breed = n02091032-Italian_greyhound, label = 12

breed = n02111889-Samoyed, label = 13

breed = n02106382-Bouvier_des_Flandres, label = 14

breed = n02108551-Tibetan_mastiff, label = 15

breed = n02093991-Irish_terrier, label = 16

breed = n02102973-Irish_water_spaniel, label = 17

breed = n02089867-Walker_hound, label = 18

breed = n02085936-Maltese_dog, label = 19

breed = n02097130-giant_schnauzer, label = 20

breed = n02096051-Airedale, label = 21